In [96]:
import sqlite3
import pandas as pd

db_conn = sqlite3.connect('../../data/large/trending_ru_yt_videos.db')
df = pd.read_sql('SELECT * FROM videos', db_conn)

df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,description,gpt_ans,gpt_time_to_ans
0,gDuslQ9avLc,17.14.11,Захар и Полина учатся экономить,Т—Ж БОГАЧ,22,2017-11-13T09:09:31.000Z,"захар и полина|""учимся экономить""|""копить день...",62408,334,190,50,"Знакомьтесь, это Захар и Полина. Вместе с ними...","Бизнес-аналитик, Менеджер по продажам, Digital...",6.309183
1,AOCJIFEA_jE,17.14.11,Биржа Мемов #29. Большой выпуск,Druzhko Show,22,2017-11-13T17:32:11.000Z,"биржа мемов|""лев шагинян""|""мемы""|""пикчи""|""друж...",330043,43841,2244,2977,"В 29 выпуске Дружко Шоу Сергей сказал, что Бир...","Анализируя текстовое описание видео, я предлаг...",5.636060
2,VAWNQDgwwOM,17.14.11,ХАЙП КЭМП - СВОЙ СРЕДИ ЧУЖИХ,Юлик,24,2017-11-13T16:11:31.000Z,"юмор|""комедия""|""влог""|""блог""|""смешно""|""смешное...",424596,49854,714,2944,http://kapitany.ru/!Yulik.cap - Факультет #Кап...,"Классы, которые могут подходить к данному текс...",4.402472
3,gknkFwKQfHg,17.14.11,Сочная кесадилья с курицей,Hochland,22,2017-11-13T06:51:10.000Z,"хохланд|""сыр""|""рецепты""|""как приготовить""|""вку...",112851,3566,122,80,*** как готовить ресторанные блюда дома,"Реставратор, Кулинария",3.606990
4,3sYvZcwzZr8,17.14.11,КЛИПЫ РОДИТЕЛЕЙ НА ШКОЛЬНЫЙ ВЫПУСКНОЙ,Совергон,24,2017-11-13T16:52:36.000Z,"Совергон|""Sovergon""|""клипы""|""родители""|""школа""...",243469,36216,631,1692,❏ КОНКУРС КАПИТАНЫ: http://kapitany.ru/!Soverg...,Аудитория видео может быть классифицирована ка...,4.333187


In [91]:
unique_tags = set()

for index, row in df.iterrows():
    tags_list = row['tags'].split('\"|\"')
    for tag in tags_list:
        unique_tags.add(tag.strip())

len(unique_tags)

190179

In [97]:
# Загружаем данные о видео (пример вашего DataFrame)
videos_df = df

# Заменяем None на пустые строки в интересующих нас колонках
videos_df['title'] = videos_df['title'].fillna('')
videos_df['tags'] = videos_df['tags'].fillna('')
videos_df['description'] = videos_df['description'].fillna('')

videos_df['title'] = videos_df['title'].str.lower()
videos_df['tags'] = videos_df['tags'].str.lower()
videos_df['description'] = videos_df['description'].str.lower()

videos_df = videos_df.drop(columns=['gpt_ans', 'gpt_time_to_ans', 'comment_count', 'likes', 'dislikes', 'views', 'publish_time', 'trending_date'])

# Загружаем данные о категориях (допустим у вас уже есть DataFrame с категориями и ключевыми словами)
conn = sqlite3.connect('../../data/professions.db')
categories_df = pd.read_sql('SELECT * FROM data', conn)

categories_df.head()

,name,category,desc,profile_codes,keywords,keywords_extention
0,Агроном,Агропромышленность,"Выращивает ягоды, овощи, фрукты, грибы, кормов...","35.02.05 - Агрономия, 35.03.03 - Агрохимия и а...","агрономия, выращивание, растения, удобрения, п...","агрономия, выращивание, растения, удобрения, п..."
1,Маркшейдер,"Архитектура, строительство и недвижимость","Исследует месторождения полезных ископаемых, п...","05.03.01 - Геология, 21.05.04 - Горное дело, 2...","маркшейдер, геодезия, исследование месторожден...","маркшейдер, геодезия, исследование месторожден..."
2,Реставратор,"Архитектура, строительство и недвижимость","Восстанавливает старинные здания, сооружения и...","07.02.01 - Архитектура, 54.02.04 - Реставрация...","реставрация, реставратор, консервация искусств...","реставрация, реставратор, консервация искусств..."
3,Слесарь по ремонту строительных машин,"Архитектура, строительство и недвижимость",Ремонтирует строительную технику<br><br>\n🔧 Ре...,23.02.03 - Техническое обслуживание и ремонт а...,"слесарь, ремонт строительных машин, техническо...","слесарь, ремонт строительных машин, техническо..."
4,Монтажник санитарно-технических систем и обору...,"Архитектура, строительство и недвижимость",Устанавливает и обслуживает системы водоснабже...,08.01.26 - Мастер по ремонту и обслуживанию ин...,"монтажник, сантехника, водоснабжение, канализа...","монтажник, сантехника, водоснабжение, канализа..."


In [98]:
# Функция для подсчёта вхождений ключевых слов каждой категории
def match_categories(row, categories_df):
    category_matches = {}
    # Объединяем title, tags и description с добавлением пробелов в начале и в конце
    tags = row['tags'].replace('\"|\"', ' ')
    combined_text = " " + f"{row['title']} {tags} {row['description']}".lower() + " "
    for _, category_row in categories_df.iterrows():
        category_name = category_row['name']
        keywords = category_row['keywords_extention'].split(', ')
        # print(keywords)
        for keyword in keywords:
            # Добавляем пробелы к каждому ключевому слову для точного поиска
            keyword_with_spaces = " " + keyword.lower() + " "
            matches = combined_text.count(keyword_with_spaces)
        if matches > 0:
            category_matches[category_name] = matches
    return category_matches if category_matches else {'Unknown': 0}

# Применяем функцию к каждой строке в DataFrame с видео
videos_df['category_matches'] = videos_df.apply(lambda row: match_categories(row, categories_df), axis=1)

In [94]:
# Создаем новые столбцы для категорий и количества вхождений
videos_df['categories'] = ""
videos_df['categories_cnt'] = ""

for idx, row in videos_df.iterrows():
    categories_str = ""
    cnt_str = ""
    # Десериализуем строку JSON обратно в словарь
    category_matches = row['category_matches']
    for category_name, count in category_matches.items():
        categories_str += category_name + ","
        cnt_str += str(count) + ","
    # Обновляем DataFrame напрямую
    videos_df.at[idx, 'categories'] = categories_str.rstrip(',')
    videos_df.at[idx, 'categories_cnt'] = cnt_str.rstrip(',')

videos_df = videos_df.drop(columns=['category_matches'])

# Сохраняем результат
conn = sqlite3.connect('prof_with_categories.db')
videos_df.to_sql('data', conn, if_exists='replace', index=False)

conn.close()  # Не забываем закрыть соединение с базой данных

In [95]:
videos_df = videos_df[videos_df['categories'] != 'Unknown']
len(videos_df)

937